In [1]:
conda env list

# conda environments:
#
                         /Users/joyboyav/Desktop/Mlops/Foundations/Mlops/End2End/env
base                     /Users/joyboyav/anaconda3
c2                    *  /Users/joyboyav/anaconda3/envs/c2
chat                     /Users/joyboyav/anaconda3/envs/chat
demo                     /Users/joyboyav/anaconda3/envs/demo
mlr                      /Users/joyboyav/anaconda3/envs/mlr
nlp                      /Users/joyboyav/anaconda3/envs/nlp
tweet                    /Users/joyboyav/anaconda3/envs/tweet
youtube                  /Users/joyboyav/anaconda3/envs/youtube


Note: you may need to restart the kernel to use updated packages.


In [5]:

from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

/Users/joyboyav/anaconda3/envs/c2/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [6]:
PINECONE_API_KEY = "6ca76a00-c508-411b-86d0-ce44394b5521" #this is from API key
PINECONE_API_ENV = "gcp-starter" #this is index key

In [17]:
from langchain.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("data/G.pdf")
data = loader.load()

In [27]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/G.pdf")
pages = loader.load()

In [28]:
len(pages)

1030

In [76]:
# split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
text_chunks = text_splitter.split_documents(pages)


In [122]:
len(text_chunks)

4133

In [109]:
#download embedding model
from langchain.embeddings import HuggingFaceEmbeddings
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

embeddings = download_hugging_face_embeddings()
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [51]:
# LEts test it
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [82]:
import pinecone

# PINECONE_API_KEY = "6ca76a00-c508-411b-86d0-ce44394b5521" #this is from API key
# PINECONE_API_ENV = "gcp-starter" #this is index key
# export PINECONE_API_KEY="6ca76a00-c508-411b-86d0-ce44394b5521" # on terminal

PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', '6ca76a00-c508-411b-86d0-ce44394b5521')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'gcp-starter')
     


In [85]:
import os
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)

In [97]:
# If you already have an index, you can load it like this
index_name = "chat-bot"
index = pc.Index(host='https://chat-bot-gdxe800.svc.gcp-starter.pinecone.io')

In [126]:
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Assuming embeddings_model is your loaded model for generating embeddings
embeddingslist = []
for doc in text_chunks:
    page_content = doc.page_content
    # Generate an embedding for the page_content
    embed = model.encode([page_content])  # Adjust based on your model's API
    embeddingslist.append(embed)

# Now, embeddings is a list of embeddings corresponding to your Document objects

# Assuming embeddings are a list of lists (nested), where each inner list should be concatenated
corrected_data_to_upload = []
for i, (text, embedding) in enumerate(zip(text_chunks, embeddingslist)):
    # Flatten the nested list structure of each embedding into a flat list
    flat_vector = [item for sublist in embedding for item in sublist] if isinstance(embedding[0], list) else embedding
    corrected_data_to_upload.append((str(i), flat_vector, {"text": text}))


In [ ]:
# export PINECONE_API_KEY='your_pinecone_api_key_here'
# export PINECONE_INDEX_NAME='your_index_name_here'

# os.environ.get('PINECONE_API_KEY', '6ca76a00-c508-411b-86d0-ce44394b5521')
# os.environ.get('PINECONE_API_ENV', 'gcp-starter')

# Set environment variables
os.environ['PINECONE_API_KEY'] = '6ca76a00-c508-411b-86d0-ce44394b5521'
os.environ['PINECONE_INDEX_NAME'] = "chat-bot"

In [127]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(text_chunks, embeddings, index_name="chat-bot")

In [128]:
query = "Who is krishna?"
docs = docsearch.similarity_search(query)
print(docs[0].page_content)

"Bhagavad-Gita As It Is" by  A.C. Bhaktivedanta Swami 
Prabhupada, courtesy of th e Bhaktivedanta Book Trust 
International, www.Krishna.com ." 
 
This book and electronic file is Copyright 1972-2004 
Bhaktivedanta Book Tr ust International, 3764 Watseka Avenue, 
Los Angeles, California 90034, USA. All rights reserved. For any 
questions, comments, corresponde nce, or to evaluate dozens of 
other books in this collection, vi sit the website of the publishers, 
www.Krishna.com .


In [ ]:
from langchain.prompts import PromptTemplate

prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""